## [Rohlik2 Lama v6 weighted](https://www.kaggle.com/code/samvelkoch/rohlik2-lama-v6-weighted)

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import requests
import joblib

import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import torch

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

'nlp' extra dependency package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


c:\Users\OWNER\Desktop\coding_project\kr\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\OWNER\Desktop\coding_project\kr\venv\Lib\site-packages\lightautoml\ml_algo\dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
c:\Users\OWNER\Desktop\coding_project\kr\venv\Lib\site-packages\lightautoml\text\embed.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
c:\Users\OWNER\Desktop\coding_project\kr\venv\Lib\site-packages\lightautoml\text\dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


In [2]:
N_THREADS = 12
N_FOLDS = 8
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 3600*100

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [3]:
train = pd.read_csv('../../data/sales_train.csv', parse_dates= ['date'])
test = pd.read_csv('../../data/sales_test.csv', parse_dates= ['date']  )
ss = pd.read_csv('../../data/solution.csv')
inventory = pd.read_csv('../../data/inventory.csv')
weights  = pd.read_csv('../../data/test_weights.csv')
calendar  = pd.read_csv('../../data/calendar.csv', parse_dates= ['date'])

In [4]:
Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

def process_calendar(df):
    """
    Обрабатывает календарный датафрейм, добавляя новые колонки:
    - days_to_holiday
    - days_to_shops_closed
    - day_after_closing
    - long_weekend
    - weekday
    """
    # Убеждаемся, что даты отсортированы
    df = df.sort_values('date').reset_index(drop=True)
    
    # 1. days_to_holiday
    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)
    
    # 2. days_to_shops_closed
    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)
    
    # 3. day_after_closing
    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    # 4. long_weekend
    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    # 5. weekday
    df['weekday'] = df['date'].dt.weekday  # 0 (понедельник) - 6 (воскресенье)
    
    return df


# Список датафреймов
dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']

# Применяем функцию ко всем датафреймам и собираем их в список
processed_dfs = [process_calendar(globals()[df]) for df in dfs]

# Конкатенируем все датафреймы в один
calendar_extended = pd.concat(processed_dfs).sort_values('date').reset_index(drop=True)

In [5]:
train_calendar = train.merge(calendar_extended, on=['date', 'warehouse'], how='left')
train_inventory = train_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')
train_data = train_inventory.merge(weights, on=['unique_id'], how='left')

test_calendar = test.merge(calendar_extended, on=['date', 'warehouse'], how='left')
test_data = test_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')

In [6]:
train_data = train_data.drop(columns=['availability'])

train_data.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,...,day_after_closing,long_weekend,weekday,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,weight
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,0.00000,0.0,0.0,0.0,...,0,0,6,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,1.925596
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,0.00000,0.0,0.0,0.0,...,1,0,1,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,1.925596
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,0.00000,0.0,0.0,0.0,...,0,0,0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,1.925596
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.20024,0.0,0.0,0.0,...,0,0,5,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,1.925596
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,0.00000,0.0,0.0,0.0,...,0,0,4,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,1.925596


In [7]:
train_data.dropna(subset=['sales'], inplace=True)

In [8]:
train_data.dtypes

unique_id                          int64
date                      datetime64[ns]
warehouse                         object
total_orders                     float64
sales                            float64
sell_price_main                  float64
type_0_discount                  float64
type_1_discount                  float64
type_2_discount                  float64
type_3_discount                  float64
type_4_discount                  float64
type_5_discount                  float64
type_6_discount                  float64
holiday_name                      object
holiday                            int64
shops_closed                       int64
winter_school_holidays             int64
school_holidays                    int64
days_to_holiday                  float64
days_to_shops_closed             float64
day_after_closing                  int32
long_weekend                       int32
weekday                            int32
product_unique_id                  int64
name            

In [9]:
train_data = train_data.sort_values(['unique_id', 'date'])
#train_data = train_data.set_index('date')
train_data.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,...,day_after_closing,long_weekend,weekday,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,weight
3910978,0,2022-07-18,Budapest_1,5289.0,3.97,710.89,0.0,0.0,0.00000,0.0,...,0,0,0,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369
3910983,0,2022-07-19,Budapest_1,5255.0,73.36,710.89,0.0,0.0,0.00000,0.0,...,0,0,1,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369
3910959,0,2022-07-20,Budapest_1,5334.0,558.09,710.89,0.0,0.0,0.45045,0.0,...,0,0,2,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369
3910961,0,2022-07-21,Budapest_1,5459.0,14.03,710.89,0.0,0.0,0.45045,0.0,...,0,0,3,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369
3910956,0,2022-07-22,Budapest_1,5461.0,558.53,710.89,0.0,0.0,0.45045,0.0,...,0,0,4,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369


In [10]:
task = Task('reg')

In [11]:
print(train_data.isna().sum())

unique_id                       0
date                            0
warehouse                       0
total_orders                    0
sales                           0
sell_price_main                 0
type_0_discount                 0
type_1_discount                 0
type_2_discount                 0
type_3_discount                 0
type_4_discount                 0
type_5_discount                 0
type_6_discount                 0
holiday_name              3844077
holiday                         0
shops_closed                    0
winter_school_holidays          0
school_holidays                 0
days_to_holiday                 0
days_to_shops_closed            0
day_after_closing               0
long_weekend                    0
weekday                         0
product_unique_id               0
name                            0
L1_category_name_en             0
L2_category_name_en             0
L3_category_name_en             0
L4_category_name_en             0
weight        

In [12]:
%%time

automl = TabularAutoML(
    task = task,
    # timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    #general_params = {'use_algos':[['linear_l2', 'lgb', 'lgb_tuned']]},
    selection_params={'mode' : 0},
    tuning_params = {'max_tuning_time': 3600},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)


CPU times: total: 62.5 ms
Wall time: 54 ms


In [13]:
%%time
out_of_fold_predictions = automl.fit_predict(
    train_data,
    roles = {
        'target': 'sales',
        #'drop': ['unique_id']
        'weights': 'weight'
    }, 
    verbose = 2
)

[22:52:20] Stdout logging level is INFO2.
[22:52:20] Task: reg

[22:52:20] Start automl preset with listed constraints:
[22:52:20] - time: 3600.00 seconds
[22:52:20] - CPU: 12 cores
[22:52:20] - memory: 16 GB

[22:52:20] Train data shape: (4007367, 30)

[22:52:39] Layer 1 train process start. Time left 3581.01 secs
[22:54:23] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[22:54:24] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[22:57:00] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[23:00:13] Time limit exceeded after calculating fold 1

[23:00:13] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -5848.602626050974
[23:00:13] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[23:00:13] Time left 3127.30 secs

[23:02:19] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[23:02:19] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_LightGBM =====
[23:07:13] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_LightGBM ===

Optimization Progress:   1%|          | 1/101 [05:43<9:33:19, 344.00s/it, best_trial=0, best_value=-1.37e+3]

[23:17:18] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[23:17:18] The set of hyperparameters {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07}
 achieve -1371.6407 mse


[23:17:18] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[23:17:19] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[23:18:10] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[23:18:53] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[23:19:37] ===== Start working with fold 3 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[23:20:20] ===== Start working with fold 4 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[23:21:03] ===== Start working with fold 5 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[23:21:47] ===== Start working with fold 6 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[23:22:30] ===== Start working with fold 7 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[23:23:13] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = -3045.868524721218
[23:23:13] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[23:23:13] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 176.86 secs


Optimization Progress:   5%|▍         | 5/101 [03:15<1:02:39, 39.16s/it, best_trial=4, best_value=-2.9e+3] 

[23:26:29] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[23:26:29] The set of hyperparameters {'max_depth': 6, 'nan_mode': 'Min', 'l2_leaf_reg': 1.9826980964985924e-05, 'min_data_in_leaf': 10}
 achieve -2901.9488 mse
[23:26:29] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...


[23:26:30] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[23:27:41] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[23:28:52] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[23:30:03] ===== Start working with fold 3 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[23:31:15] ===== Start working with fold 4 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[23:32:26] ===== Start working with fold 5 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[23:33:38] ===== Start working with fold 6 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[23:34:49] ===== Start working with fold 7 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[23:36:00] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = -2770.8942126095276
[23:36:00] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[23:36:00] Time left 980.22 secs

[23:36:00] Time limit exceeded in one of the tasks. AutoML will blend level 1 models.

[23:36:00] Lay

In [14]:
joblib.dump(automl, 'model.pkl')
loaded_model = joblib.load('model.pkl')

In [15]:
test_predictions = loaded_model.predict(test_data)

In [16]:
out_of_fold_predictions = loaded_model.fit_predict(
    train_data,
    roles = {
        'target': 'sales',
        #'drop': ['unique_id']
        'weights': 'weight'
    }, 
    verbose = 2
)

[23:36:17] Stdout logging level is INFO2.
[23:36:17] Task: reg

[23:36:17] Start automl preset with listed constraints:
[23:36:17] - time: 3600.00 seconds
[23:36:17] - CPU: 12 cores
[23:36:17] - memory: 16 GB

[23:36:17] Train data shape: (4007367, 30)

[23:36:36] Layer 1 train process start. Time left 3581.19 secs
[23:38:14] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[23:38:15] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[23:40:38] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[23:43:28] Time limit exceeded after calculating fold 1

[23:43:28] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -5848.608184528893
[23:43:28] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[23:43:28] Time left 3169.07 secs

[23:45:31] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[23:45:31] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_LightGBM =====
[23:50:28] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_LightGBM ===

Optimization Progress:   1%|          | 1/101 [05:46<9:38:06, 346.86s/it, best_trial=0, best_value=-1.37e+3]

[00:00:57] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[00:00:57] The set of hyperparameters {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07}
 achieve -1371.6407 mse


[00:00:58] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[00:00:59] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[00:01:42] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[00:02:25] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[00:03:08] ===== Start working with fold 3 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[00:03:52] ===== Start working with fold 4 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[00:04:35] ===== Start working with fold 5 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[00:05:18] ===== Start working with fold 6 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[00:06:02] ===== Start working with fold 7 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[00:06:44] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = -3044.9115579232885
[00:06:44] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[00:06:44] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 202.19 secs


Optimization Progress:   6%|▌         | 6/101 [04:05<1:04:43, 40.87s/it, best_trial=5, best_value=-2.89e+3]

[00:10:50] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[00:10:50] The set of hyperparameters {'max_depth': 6, 'nan_mode': 'Min', 'l2_leaf_reg': 0.0021465011216654484, 'min_data_in_leaf': 1}
 achieve -2891.6232 mse
[00:10:50] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...


[00:10:51] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[00:12:02] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[00:13:13] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[00:14:24] ===== Start working with fold 3 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[00:15:35] ===== Start working with fold 4 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[00:16:47] ===== Start working with fold 5 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[00:17:58] ===== Start working with fold 6 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[00:19:09] ===== Start working with fold 7 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[00:20:12] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = -2782.9161553302147
[00:20:12] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[00:20:12] Time left 965.24 secs

[00:20:12] Time limit exceeded in one of the tasks. AutoML will blend level 1 models.

[00:20:12] Lay

In [17]:
test_predictions.data

array([[ 18.600853],
       [ 78.39125 ],
       [ 82.22325 ],
       ...,
       [113.27179 ],
       [598.99634 ],
       [ 28.053886]], dtype=float32)

In [18]:
submission = pd.DataFrame({
    'id': ss.id.values,
    'sales_hat': test_predictions.data[:, 0],
})

In [19]:
submission

,id,sales_hat
0,1226_2024-06-03,18.600853
1,1226_2024-06-11,78.391251
2,1226_2024-06-13,82.223251
3,1226_2024-06-15,69.548752
4,1226_2024-06-09,17.015873
...,...,...
47016,4572_2024-06-03,52.218864
47017,3735_2024-06-04,102.074844
47018,3735_2024-06-03,113.271790
47019,2129_2024-06-03,598.996338


In [20]:
submission.to_csv('lama_v6_weighted1.csv', index = False)

In [30]:
print(f'MAE score: {mean_absolute_error(train_data["sales"].values, out_of_fold_predictions.data[:,0])}')


ValueError: Input contains NaN.

In [29]:
out_of_fold_predictions

array([[       nan],
       [       nan],
       [411.33035 ],
       ...,
       [ 20.722275],
       [       nan],
       [ 15.26353 ]], dtype=float32)